In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# Connect to workspace
ml_client = MLClient(
    DefaultAzureCredential(),
    subscription_id="2d3821fb-1fc0-4f0e-b514-0bdca749349b",
    resource_group_name="ml-rg-demo",
    workspace_name="ml-workspace-demo"
)

print("✅ Connected to workspace:", ml_client.workspace_name)


In [ ]:
from azure.ai.ml.entities import Data

my_data = Data(
    name="walmart-data",
    description="Demo dataset for walmart prediction",
    path="Walmart.csv",  # relative to notebook folder
    type="uri_file"
)

ml_client.data.create_or_update(my_data)
print("✅ Dataset registered")

In [ ]:
from azure.ai.ml.entities import AmlCompute

cpu_cluster = AmlCompute(
    name="cpu-cluster",              # cluster name
    size="Standard_D2s_v3",          # VM type (4 CPU, 14 GB RAM)
    min_instances=0,                 # scale down when idle
    max_instances=2,                 # up to 2 nodes
    idle_time_before_scale_down=120  # seconds
)

ml_client.compute.begin_create_or_update(cpu_cluster).result()
print("✅ cpu-cluster created")


In [ ]:
from azure.ai.ml.entities import Environment

env = Environment(
    name="walmart-env",
    version="2",   # must include version
    conda_file="conda.yml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04"
)
ml_client.environments.create_or_update(env)

In [ ]:
from azure.ai.ml import command, Input

job = command(
    code=".",  # folder containing train.py
    command="python train.py --data ${{inputs.data}} --n_estimators 200",
    inputs={"data": Input(type="uri_file", path="walmart-data:1")},
    environment="walmart-env:2",
    compute="cpu-cluster",
    experiment_name="walmart-mlflow-training"
)

returned_job = ml_client.jobs.create_or_update(job)
ml_client.jobs.stream(returned_job.name)


In [ ]:
from azure.ai.ml.entities import Model

model = Model(
    path=f"azureml://jobs/{returned_job.name}/outputs/artifacts/model.pkl",
    name="walmart-rf-model",
    description="Random Forest model trained on Walmart dataset"
)

ml_client.models.create_or_update(model)
print("✅ Model registered")
